<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [5]:
import pandas as pd
import matplotlib.pyplot as plt 
plt.style.use('seaborn-pastel')
import seaborn as sns 
import numpy as np 

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.metrics import f1_score

import time


In [6]:
df = pd.read_csv('/Users/dzumahonazizov/Desktop/toxic_comments.csv')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [7]:
nltk.download('punkt')
nltk.download('wordnet')

stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dzumahonazizov/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/dzumahonazizov/nltk_data...


Создадим функцию для токенизации, лемматизации и очищения текста

In [10]:
from tqdm.notebook import tqdm
tqdm.pandas()

import spacy

nlp = spacy.load("en_core_web_sm")

def clear_text(text):
    return re.sub(r'[^a-zA-Z ]', ' ', text.lower())

def lemmatize_with_spacy(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_.strip() for token in doc])
    return lemmatized_text

start_time = time.time()
df['lemm_text'] = df['text'].apply(clear_text).progress_apply(lemmatize_with_spacy)
print("Время:", time.time() - start_time)

df['lemm_text']

  0%|          | 0/159292 [00:00<?, ?it/s]

Время: 2849.3866000175476


0         explanation why the edit make under my usernam...
1         d aww  he match this background colour I m see...
2         hey man  I m really not try to edit war  it s ...
3          more I can t make any real suggestion on impr...
4         you  sir  be my hero  any chance you remember ...
                                ...                        
159287     and for the second time of ask  when your vie...
159288    you should be ashamed of yourself  that be a h...
159289    spitzer  umm  there s no actual article for pr...
159290    and it look like it be actually you who put on...
159291     and  I really don t think you understand  I c...
Name: lemm_text, Length: 159292, dtype: object

Делим датасет на тестовую и тренировочную выборки, размер тестовой выборки - 20% от общих данных:

In [11]:
train_features, test_features, train_target, test_target = train_test_split(
    df.drop('toxic', axis=1),
    df['toxic'],
    test_size=0.2,
    random_state=123,
    stratify=df['toxic']
)

corpus_train = train_features['lemm_text']
corpus_test = test_features['lemm_text']
corpus_train

141849     the article claim that the town s name be spe...
38990      request creation of  denise milani  I m not a...
88766      the  hour block that you place on my account ...
137065     I would be greatful if  could explain how a c...
31942      jdorney really need to grow up here  first of...
                                ...                        
106702     excuse I  unless you can provide I source to ...
144562    incorrect  you all have make mistake here  sea...
3336      complete bull  oakland hasn t make the playoff...
73934                   sure  but you ll miss my rfa  talk 
73485     deletion nomination  thank for support john pi...
Name: lemm_text, Length: 127433, dtype: object

Создаем мешок слов и вычисляем TF-IDF для корпуса текстов

In [12]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test) 

print("Размер матрицы:", tf_idf_train.shape) 
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (127433, 132485)
Размер матрицы: (31859, 132485)


## Обучение

Создадим функцию с GridSearch и подберем 3 модели с наилучшими параметрами и показателями

In [13]:
def train_model(model, parameters, scoring='f1', cv=3):
    model_grid = GridSearchCV(
        estimator=model,
        param_grid=parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cv,
        verbose=2
    )
    
    model_grid.fit(tf_idf_train, train_target)
    
    return model_grid.best_params_, model_grid.best_score_, model_grid.best_estimator_

Линейная регрессия

In [14]:
param_grid_lr = {
    "penalty": ['l1', 'l2'],
    "class_weight": [None, 'balanced'],
}

logistic_regression = LogisticRegression(max_iter=1000, solver='liblinear')

best_params_lr, f1_lr, best_model_lr = train_model(logistic_regression, param_grid_lr)

print(f'Лучшие параметры Логистической регрессии: {best_params_lr}')
print(f'f1: {f1_lr:.2f}')

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Лучшие параметры Логистической регрессии: {'class_weight': None, 'penalty': 'l1'}
f1: 0.77


Случайный лес

In [15]:
param_grid_rf = {
    "n_estimators": [10, 100, 10],
    "max_depth": [2, 12, 2],
    "class_weight": [None, 'balanced']
}

random_forest = RandomForestClassifier(random_state=123)

best_params_rf, f1_rf, best_model_rf = train_model(random_forest, param_grid_rf)

print(f'Лучшие параметры для Случайного леса: {best_params_rf}')
print(f'F1 score for Random Forest: {f1_rf:.2f}')


Fitting 3 folds for each of 18 candidates, totalling 54 fits
Лучшие параметры для Случайного леса: {'class_weight': 'balanced', 'max_depth': 12, 'n_estimators': 100}
F1 score for Random Forest: 0.36


LGBM

In [16]:
param_grid_lgb = {
    'boosting_type': ['gbdt'],
    'max_depth': [5, 7],
    'learning_rate': [0.1],
    'class_weight': [None, 'balanced']
}

lgb_classifier = lgb.LGBMClassifier(random_state=123)

best_params_lgb, f1_lgb, best_model_lgb = train_model(lgb_classifier, param_grid_lgb)

print(f'Лучшие параметры для LGBM: {best_params_lgb}')
print(f'F1 score for LGBMClassifier: {f1_lgb:.2f}')

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Лучшие параметры для LGBM: {'boosting_type': 'gbdt', 'class_weight': 'balanced', 'learning_rate': 0.1, 'max_depth': 7}
F1 score for LGBMClassifier: 0.72


## Выводы

In [17]:
results = []
results.append({'Model': 'LightGBM', 'f1': f1_lgb})
results.append({'Model': 'LinearRegression', 'f1': f1_lr})
results.append({'Model': 'RandomForest', 'f1': f1_rf})

df_results = pd.DataFrame(results)
df_results_sorted = df_results.sort_values(by='f1', ignore_index=True, ascending=False)
display(df_results_sorted)

,Model,f1
0,LinearRegression,0.767167
1,LightGBM,0.724762
2,RandomForest,0.364486


In [18]:
best_model = best_model_lr

predictions = best_model.predict(tf_idf_test)
print('F1:', f1_score(test_target, predictions))

F1: 0.7807598466364588
[CV] END ..................class_weight=balanced, penalty=l1; total time=   4.0s
[CV] END ....class_weight=None, max_depth=2, n_estimators=10; total time=   2.0s
[CV] END ...class_weight=None, max_depth=12, n_estimators=10; total time=   6.2s
[CV] END ..class_weight=None, max_depth=12, n_estimators=100; total time=  52.9s
[CV] END boosting_type=gbdt, class_weight=None, learning_rate=0.1, max_depth=7; total time=  59.5s
[CV] END ......................class_weight=None, penalty=l2; total time=   3.4s
[CV] END ...class_weight=None, max_depth=2, n_estimators=100; total time=  11.6s
[CV] END ...class_weight=None, max_depth=12, n_estimators=10; total time=   6.3s
[CV] END ....class_weight=None, max_depth=2, n_estimators=10; total time=   1.6s
[CV] END class_weight=balanced, max_depth=2, n_estimators=10; total time=   1.4s
[CV] END class_weight=balanced, max_depth=2, n_estimators=100; total time=  11.1s
[CV] END class_weight=balanced, max_depth=12, n_estimators=10; tota

Модель LinearRegression показывает результат 0.78 что выше 0.75, значит результат нас устраивает, мы проделали отличную работу!

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны